Visualisation of class spread in dataset

In [1]:
import pandas
TRAINING_SET = pandas.read_csv('train.csv')

In [9]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

histograms = {}

for index in range(2,8):
    #histograms.append(np.histogram(TRAINING_SET.iloc[:,index], bins=2))
    histograms.update({TRAINING_SET.iloc[:,index].name : np.histogram(TRAINING_SET.iloc[:,index], bins=2)[0]})

In [10]:
histograms

{'toxic': array([144277,  15294], dtype=int64),
 'severe_toxic': array([157976,   1595], dtype=int64),
 'obscene': array([151122,   8449], dtype=int64),
 'threat': array([159093,    478], dtype=int64),
 'insult': array([151694,   7877], dtype=int64),
 'identity_hate': array([158166,   1405], dtype=int64)}